In [5]:
import pandas as pd
from datetime import datetime

# Load the dataset
file_path = "dataset.csv"
data = pd.read_csv(file_path)

def find_patient(patient_id):
    global data  # Declare `data` as global
    # Check if the patient exists
    result = data[data["PatientID"] == patient_id]
    if not result.empty:
        print("Patient Found:")
        print(result)
    else:
        print("Patient not found. Adding new patient details.")
        add_patient()

def add_patient():
    global data  # Declare `data` as global
    # Get the next PatientID
    next_patient_id = data["PatientID"].max() + 1 if not data.empty else 1
    print(f"Assigning new PatientID: {next_patient_id}")
    
    # Prompt user for other details
    details = {
        "PatientID": next_patient_id,
        "State": input("Enter State: "),
        "Sex": input("Enter Sex (Male/Female): "),
        "GeneralHealth": input("Enter General Health: "),
        "AgeCategory": input("Enter Age Category: "),
        "HeightInMeters": float(input("Enter Height in Meters: ")),
        "WeightInKilograms": float(input("Enter Weight in Kilograms: ")),
        "BMI": float(input("Enter BMI: ")),
        "Disease": input("Enter Diseases (comma-separated): "),
        "JoinedDate": input("Enter Joined Date (YYYY-MM-DD): "),
        "CheckoutDate": input("Enter Checkout Date (YYYY-MM-DD): "),
        "Insurance": input("Has Insurance (Yes/No): ")
    }
    
    # Append to the dataset using pd.concat
    new_row = pd.DataFrame([details])  # Convert dictionary to DataFrame
    data = pd.concat([data, new_row], ignore_index=True)
    print("Patient details added successfully.")

    # Save the dataset
    data.to_csv(file_path, index=False)
    print("Dataset updated.")

# Main program loop
while True:
    try:
        patient_id = int(input("Enter Patient ID to search: "))
        find_patient(patient_id)
    except ValueError:
        print("Invalid Patient ID. Please enter a valid integer.")
    
    cont = input("Do you want to continue? (yes/no): ").strip().lower()
    if cont != 'yes':
        break


Enter Patient ID to search:  69


Patient Found:
    PatientID        State   Sex GeneralHealth   AgeCategory  HeightInMeters  \
68         69  West Bengal  Male          Fair  Age 50 to 54             1.8   

    WeightInKilograms    BMI                              Disease  JoinedDate  \
68          94.800003  29.15  HadArthritis, ChestScan, HIVTesting  2020-02-11   

   CheckoutDate Insurance  
68   2020-03-01        No  


Do you want to continue? (yes/no):  yes
Enter Patient ID to search:  123654987


Patient not found. Adding new patient details.
Assigning new PatientID: 237631


Enter State:  Telangana
Enter Sex (Male/Female):  MAle
Enter General Health:  Fair
Enter Age Category:  18-20
Enter Height in Meters:  1.68
Enter Weight in Kilograms:  69
Enter BMI:  1.43
Enter Diseases (comma-separated):  Stroke
Enter Joined Date (YYYY-MM-DD):  2024-06-04
Enter Checkout Date (YYYY-MM-DD):  2024-10-04
Has Insurance (Yes/No):  Yes


Patient details added successfully.
Dataset updated.


Do you want to continue? (yes/no):  no


In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the dataset
file_path = "dataset.csv"
data = pd.read_csv(file_path)

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def train_disease_model():
    global model

    # Handle missing values
    data["Disease"] = data["Disease"].fillna("")  # Replace NaN with empty string
    data["Disease"] = data["Disease"].astype(str)  # Convert all Disease values to string

    # Label encode the AgeCategory column
    le = LabelEncoder()
    data["AgeCategory"] = le.fit_transform(data["AgeCategory"])

    # Prepare features and labels
    features = data[["AgeCategory", "BMI", "HeightInMeters", "WeightInKilograms"]]

    # Example: Using "Diabetes" as a condition to predict
    labels = data["Disease"].apply(lambda x: 1 if "Diabetes" in x else 0)  # Binary label

    # Train the model
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    
    # Evaluate the model
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Model Trained! Accuracy: {accuracy:.2f}")

# Call the function to train the model
train_disease_model()


# Function to predict disease for a new patient
def predict_disease(new_patient_details):
    global model
    prediction = model.predict([new_patient_details])
    disease_status = "Likely has Diabetes" if prediction[0] == 1 else "No Diabetes Detected"
    return disease_status

# Train the disease model at the start
train_disease_model()

def find_patient(patient_id):
    global data
    # Check if the patient exists
    result = data[data["PatientID"] == patient_id]
    if not result.empty:
        print("Patient Found:")
        print(result)
    else:
        print("Patient not found. Adding new patient details.")
        add_patient()

def add_patient():
    global data
    # Get the next PatientID
    next_patient_id = data["PatientID"].max() + 1 if not data.empty else 1
    print(f"Assigning new PatientID: {next_patient_id}")
    
    # Prompt user for other details
    new_details = {
        "PatientID": next_patient_id,
        "State": input("Enter State: "),
        "Sex": input("Enter Sex (Male/Female): "),
        "GeneralHealth": input("Enter General Health: "),
        "AgeCategory": int(input("Enter Age Category: ")),  # Assuming it's numerical
        "HeightInMeters": float(input("Enter Height in Meters: ")),
        "WeightInKilograms": float(input("Enter Weight in Kilograms: ")),
        "BMI": float(input("Enter BMI: ")),
        "Disease": input("Enter Diseases (comma-separated): "),
        "JoinedDate": input("Enter Joined Date (YYYY-MM-DD): "),
        "CheckoutDate": input("Enter Checkout Date (YYYY-MM-DD): "),
        "Insurance": input("Has Insurance (Yes/No): ")
    }
    
    # Predict disease
    new_patient_features = [
        new_details["AgeCategory"],
        new_details["BMI"],
        new_details["HeightInMeters"],
        new_details["WeightInKilograms"]
    ]
    prediction = predict_disease(new_patient_features)
    print(f"Disease Prediction for New Patient: {prediction}")
    
    # Append new patient to the dataset
    data = pd.concat([data, pd.DataFrame([new_details])], ignore_index=True)
    print("Patient details added successfully.")

    # Save the updated dataset
    data.to_csv(file_path, index=False)
    print("Dataset updated.")

# Main program loop
while True:
    try:
        patient_id = int(input("Enter Patient ID to search: "))
        find_patient(patient_id)
    except ValueError:
        print("Invalid Patient ID. Please enter a valid integer.")
    
    cont = input("Do you want to continue? (yes/no): ").strip().lower()
    if cont != 'yes':
        break


Model Trained! Accuracy: 1.00
Model Trained! Accuracy: 1.00


Enter Patient ID to search:  69


Patient Found:
    PatientID        State   Sex GeneralHealth  AgeCategory  HeightInMeters  \
68         69  West Bengal  Male          Fair            7             1.8   

    WeightInKilograms    BMI                              Disease  JoinedDate  \
68          94.800003  29.15  HadArthritis, ChestScan, HIVTesting  2020-02-11   

   CheckoutDate Insurance  
68   2020-03-01        No  


Do you want to continue? (yes/no):  yes
Enter Patient ID to search:  236598741


Patient not found. Adding new patient details.
Assigning new PatientID: 237632


...

C:\Users\KGRCET\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Disease Prediction for New Patient: No Diabetes Detected
Patient details added successfully.
Dataset updated.


Do you want to continue? (yes/no):  YEs
Enter Patient ID to search:  69


Patient Found:
    PatientID        State   Sex GeneralHealth  AgeCategory  HeightInMeters  \
68         69  West Bengal  Male          Fair            7             1.8   

    WeightInKilograms    BMI                              Disease  JoinedDate  \
68          94.800003  29.15  HadArthritis, ChestScan, HIVTesting  2020-02-11   

   CheckoutDate Insurance  
68   2020-03-01        No  


Do you want to continue? (yes/no):  no
